# Demographic 사용 추천

In [1]:
import numpy as np
print(f'NumPy v{np.__version__}')

import pandas as pd
print(f'pandas v{pd.__version__}')

import sklearn
print(f'scikit-learn v{sklearn.__version__}')

# Only for specifying versions
import sys; print(f'Python v{sys.version}')

NumPy v1.25.0
pandas v1.5.3
scikit-learn v1.2.0
Python v3.9.16 (main, May 17 2023, 17:49:16) [MSC v.1916 64 bit (AMD64)]


## 2.4 사용자 집단별 추천

In [2]:
# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('../Data/u.user', sep='|', names=u_cols, encoding='latin-1')

i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('../Data/u.item', sep='|', names=i_cols, encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../Data/u.data', sep='\t', names=r_cols, encoding='latin-1')


# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# timestamp 제거 
ratings.drop('timestamp', axis='columns', inplace=True)

In [3]:
# train, test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

x_train.head()

,user_id,movie_id,rating
68016,450,230,4
71644,886,54,3
10494,200,231,4
5440,159,237,3
92133,496,1041,1


In [4]:
# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')
rating_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1673,1674,1675,1677,1678,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,NaN,3.0,3.0,NaN,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


이전의 best-seller 모델을 테스트해보자. 사용자의 특성에 관계없이 같은 값으로 예측한다.

In [5]:
train_mean = x_train.groupby(['movie_id'])['rating'].mean()
train_mean.head()

movie_id
1    3.855114
2    3.188889
3    3.015873
4    3.618750
5    3.216667
Name: rating, dtype: float64

In [6]:
# 전체 평균으로 예측치를 계산하는 기본 모델
def best_seller(user_id, movie_id):
    try:
        rating = train_mean[movie_id]
    except:
        rating = 3.0    # 3.0 대신 train_mean.mean()을 쓰면 점수가 살짝 올라간다.
    return rating

score(best_seller)

1.0214539147731656

사용자의 성별에 따라 예측을 달리 해보자.

In [7]:
# Full matrix를 사용자 데이터와 merge
merged_ratings = pd.merge(x_train, users)
users.set_index('user_id', inplace=True)
merged_ratings.head()

,user_id,movie_id,rating,age,sex,occupation,zip_code
0,450,230,4,35,F,educator,11758
1,450,234,3,35,F,educator,11758
2,450,25,3,35,F,educator,11758
3,450,465,4,35,F,educator,11758
4,450,154,3,35,F,educator,11758


In [8]:
# gender별 평점평균 계산
g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()
g_mean.head(8)

movie_id  sex
1         F      3.833333
          M      3.863281
2         F      3.545455
          M      3.139241
3         F      2.666667
          M      3.098039
4         F      3.576923
          M      3.626866
Name: rating, dtype: float64

In [9]:
###### Gender기준 추천 ######
# gender별 평균을 예측치로 돌려주는 함수 
def cf_gender(user_id, movie_id):
    if movie_id in rating_matrix:
        gender = users.loc[user_id]['sex']
        if gender in g_mean[movie_id]:
            gender_rating = g_mean[movie_id][gender]
        else:
            gender_rating = 3.0     # train_mean[movie_id]
    else:
        gender_rating = 3.0     # train_mean.mean()
    return gender_rating

score(cf_gender)

1.0306573039924316

best-seller 모델에 비해 사용자의 성별에 따른 추천은 정확도를 개선하지 못했다.

## 연습문제

Q. 사용자의 직업(`'occupation'`)에 따라 집단을 나누어서 예측값을 구하는 함수를 만들고 이의 정확도를 계산하는 코드를 작성하라.

In [10]:
users['occupation'].unique()

array(['technician', 'other', 'writer', 'executive', 'administrator',
       'student', 'lawyer', 'educator', 'scientist', 'entertainment',
       'programmer', 'librarian', 'homemaker', 'artist', 'engineer',
       'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor'],
      dtype=object)

In [11]:
o_mean = merged_ratings[['movie_id', 'occupation', 'rating']].groupby(['movie_id', 'occupation'])['rating'].mean()

def cf_occup(user_id, movie_id):
    if movie_id in rating_matrix:
        occup = users.loc[user_id]['occupation']
        if occup in o_mean[movie_id]:
            rating = o_mean[movie_id][occup]
        else:
            rating = 3.0    # train_mean[movie_id]를 쓰면 오히려 점수가 낮아진다.
    else:
        rating = 3.0    # train_mean.mean()을 쓰는 것과 별 차이가 없다.
    return rating

score(cf_occup)

1.115557049712179

best-seller보다 (소수점 첫째 자리만큼) 나쁜 성능을 보이는 것을 볼 수 있다.

Q. 사용자의 성별과 직업을 동시에 고려한 집단을 나누어서 예측값을 구하는 함수를 만들고 이의 정확도를 계산하는 코드를 작성하라.

In [12]:
go_mean = merged_ratings[['movie_id', 'sex', 'occupation', 'rating']].groupby(['movie_id', 'sex', 'occupation'])['rating'].mean()
go_mean.head(20)

movie_id  sex  occupation   
1         F    administrator    4.142857
               artist           4.400000
               educator         3.444444
               engineer         4.000000
               entertainment    4.500000
               executive        3.000000
               healthcare       2.250000
               homemaker        5.000000
               librarian        3.700000
               marketing        3.000000
               none             4.000000
               other            4.000000
               programmer       4.000000
               salesman         4.000000
               scientist        3.500000
               student          3.863636
               technician       4.000000
               writer           4.142857
          M    administrator    4.000000
               artist           4.400000
Name: rating, dtype: float64

In [13]:
def cf_go(user_id, movie_id):
    if movie_id in rating_matrix:
        gender = users.loc[user_id]['sex']
        occup = users.loc[user_id]['occupation']

        if gender in go_mean[movie_id]:
            if occup in go_mean[movie_id][gender]:
                rating = go_mean[movie_id][gender][occup]
            else:
                rating = 3.0
        else:
            rating = 3.0     # train_mean[movie_id]
    
    else:
        rating = 3.0     # train_mean.mean()

    return rating

score(cf_go)

1.1375149342466373

best-seller와 성별 모델을 비교했을 때와 같이, 직업 모델과 직업-성별 모델은 큰 차이가 없는 것을 알 수 있다.